In [90]:
import pickle
ss = {"sp" : [1, 2, 3], "f0" : [4,5,6] }
dd = { "v" : [ss]*5}
# d = { "val" : dd, "key" : ss }
print(dd['v'][0])
# afile = open(r'd.pkl', 'wb')

# pickle.dump(d, afile)
# afile.close()

{'sp': [1, 2, 3], 'f0': [4, 5, 6]}


In [19]:
print(d['val']['sp'])
foo = pickle.load(open("d.pkl", "rb"))
print(foo['val']['sp'])


[1, 2, 3]
[1, 2, 3]


In [146]:
import os
import soundfile as sf
import pyworld as pw
import matplotlib.pyplot as plt

aiueo = ['a','i','u','e','o']

all_data = []
note_step=0.25
frame_period = 10.0 # millisecond
def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
    
for x in os.walk('data'):

    for who in x[1]:
        rangePath = os.path.join('data',who,"pRange.pkl")
        [min_note,max_note] = pickle.load(open(rangePath, "rb"))
        print(min_note,max_note)
        for v in aiueo:
            path = os.path.join('data',who,v)
            print(path)
            x,fs = sf.read(path+'.wav')
            x = (x[:,0] + x[:,1])/2 
            x = x/max(x)*0.5
            f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)
            f0 = f0[1:-1]
            sp = sp[1:-1,:]
            file_cnt = 2
            while os.path.isfile(path+str(file_cnt)+'.wav'):
                print(path+str(file_cnt)+'.wav')
                x2,_ = sf.read(path+str(file_cnt)+'.wav')
                x2 = (x2[:,0] + x2[:,1])/2 # stereo to mono
                f02, sp2,_ = pw.wav2world(x2, fs, frame_period =  frame_period)    # use default options
                
                f0_note =np.concatenate((f0_note,f02[1:-1]), axis=0)
                sp =np.concatenate((sp,sp2), axis=0)
                
                file_cnt +=1
            valid_idx = np.where(f0!=0)
            f0 =  f0[valid_idx]
            sp =  sp[valid_idx]
            idx = get_f0_index_tbl(f0,min_note,max_note,note_step)  
#             f0 =  f0[idx]
            sp =  sp[idx,:]
            lin_f0 = np.linspace(min_note, max_note,num=len(idx))
            _y = pw.synthesize(lin_f0, sp, ap[:len(idx)], fs, frame_period)
            f02, sp2,_ = pw.wav2world(_y, fs, frame_period =  frame_period)
            key_label = {"sp" : f02, "f0" :sp2}
            all_data.append(key_label)
            

            
#             f0_note = 12*np.log2(f0/440)+49
#             plt.plot(f0)
#             plt.show()
            
    #         sorted_note = np.array(sorted(i for i in f0_note if i < note_upper+1))
    #         del_note = abs(sorted_note[1:] - sorted_note[:-1])
outfile = open(r'train_data/d.pkl', 'wb')
pickle.dump(all_data, outfile)
print('file write to train_data/d.pkl')
outfile.close()          
            

32.0 47.0
data\LISA\a
data\LISA\i
data\LISA\u
data\LISA\e
data\LISA\o
27.0 39.0
data\士峰\a
data\士峰\i
data\士峰\u
data\士峰\e
data\士峰\o
28.0 39.0
data\子齊\a
data\子齊\i
data\子齊\u
data\子齊\e
data\子齊\o
28.0 41.0
data\我\a
data\我\i
data\我\i2.wav
data\我\u
data\我\e
data\我\o
27.0 39.0
data\漢文\a
data\漢文\i
data\漢文\u
data\漢文\e
data\漢文\o
28.0 39.0
data\長霖\a
data\長霖\i
data\長霖\u
data\長霖\e
data\長霖\o
file write to train_data/d.pkl


In [143]:
outfile = open(r'train_data/d.pkl', 'wb')
pickle.dump(all_data, outfile)
print('file write to train_data/d.pkl')
outfile.close()          
            

file write to train_data/d.pkl


In [144]:
print(all_data[0]['f0'])

[[3.67630541e-04 3.67765260e-04 3.68199341e-04 ... 3.03416174e-12
  3.03356299e-12 3.03335575e-12]
 [7.21955118e-01 7.27038243e-01 7.39160177e-01 ... 2.47102588e-10
  2.47184292e-10 2.47266410e-10]
 [1.68060795e+00 1.69669017e+00 1.73689298e+00 ... 2.30595534e-10
  2.29999099e-10 2.29782188e-10]
 ...
 [4.68460549e+00 4.69353304e+00 4.71885431e+00 ... 5.87362730e-09
  5.89037604e-09 5.89536631e-09]
 [4.65167749e+00 4.66174276e+00 4.69055038e+00 ... 2.14214553e-08
  2.13710111e-08 2.13558182e-08]
 [6.43532706e-01 6.44139405e-01 6.45967557e-01 ... 8.92702925e-09
  8.89206450e-09 8.87913843e-09]]


In [ ]:
from termcolor import colored
import sys
import numpy as np
import os.path
import soundfile as sf
# import librosa
import pyworld as pw

#28 38 9 男
#28 38 9 女

frame_period = 10.0 # millisecond

aiueo = ['a','i','u','e','o']
for i in range(len(aiueo)):
    wavname = 'data/'+aiueo[i]+'.wav'
    if not os.path.isfile(wavname):
        print(colored('錯誤:找不到音檔','red'),wavname)
        continue

    x, fs = sf.read(wavname)
    x = (x[:,0] + x[:,1])/2 # stereo to mono

    num_of_frames = int(np.floor(len(x)/samples_per_frame))    
    f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options


#   pitch judgement
    f0_note = 12*np.log2(f0[1:-1]/440)+49
    min_his = min(np.histogram(f0_note, bins=13)[0])

In [132]:
x, fs = sf.read('a.wav')

try:
    x = (x[:,0] + x[:,1])/2
    x = x/max(x)*0.3
except:
    pass
frame_period = 10
f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options
f0 = f0[1:]
sp = sp[1:]
ap = ap[1:]

In [133]:
#return f0 index array with len (max_note-min_note)/note_step+1

def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
    

tbl = get_f0_index_tbl(f0,28,38)
    
        
    

In [135]:
print(tbl)

[ 40  44  46  50  52  55  57  65  70  73  77  86  95  98 102 113 117 121
 125 129 135 144 149 158 165 175 183 192 196 201 215 222 233 238 245 253
 257 262 289 301 320]


In [100]:
test = [3.3,4.4,2.2,7.7]
idx = np.argsort(test)
val = np.sort(test)

print(idx,val)
    

[2 0 1 3] [2.2 3.3 4.4 7.7]


In [120]:
tmp = []
for i in range(10):
    tmp.append(i)
print(tmp)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
